## Descriptive Statistics

In [0]:
silver_df = spark.table("default.silver_movies")

silver_df.describe(
    ["budget", "revenue", "imdb_rating_clean"]
).show()


+-------+-----------------+--------------------+------------------+
|summary|           budget|             revenue| imdb_rating_clean|
+-------+-----------------+--------------------+------------------+
|  count|               34|                  34|                33|
|   mean|      2264.917059|4432.864705882352...| 7.912121212121213|
| stddev|11970.66056370616|   17062.49592756319|1.2476872544225517|
|    min|             1.00|    3.10000000000000|               1.9|
|    max|         70000.00|100000.0000000000...|               9.3|
+-------+-----------------+--------------------+------------------+



## Hypothesis Testing

In [0]:
#Do higher-rated movies earn more revenue?
from pyspark.sql.functions import when, avg

rating_test = (
    silver_df
    .withColumn(
        "rating_group",
        when(silver_df.imdb_rating_clean >= 7, "High Rated")
        .otherwise("Low Rated")
    )
    .groupBy("rating_group")
    .agg(
        avg("revenue").alias("avg_revenue"),
        avg("budget").alias("avg_budget")
    )
)

rating_test.show()


+------------+--------------------+-----------+
|rating_group|         avg_revenue| avg_budget|
+------------+--------------------+-----------+
|  High Rated|5138.724137931034...|2633.540690|
|   Low Rated|338.8799999999999...| 126.900000|
+------------+--------------------+-----------+



## Correlation Analysis

In [0]:
silver_df.stat.corr("budget", "revenue")


0.9919008555418434

In [0]:
silver_df.stat.corr("imdb_rating_clean", "revenue")


0.08064662047046421

## Feature Engineering

In [0]:
from pyspark.sql.functions import log, col, year

features_df = (
    silver_df
    .withColumn("budget_log", log(col("budget") + 1))
    .withColumn("revenue_log", log(col("revenue") + 1))
    .withColumn("release_year_num", col("release_year").cast("int"))
    .withColumn("rating_bucket",
        when(col("imdb_rating_clean") >= 7, 1).otherwise(0)
    )
)

features_df.select(
    "budget_log",
    "revenue_log",
    "release_year_num",
    "rating_bucket"
).show(5)


+------------------+------------------+----------------+-------------+
|        budget_log|       revenue_log|release_year_num|rating_bucket|
+------------------+------------------+----------------+-------------+
|1.8718021769015913|2.5649493574615367|            2022|            1|
| 5.993961427306569| 7.601402334583733|            1995|            1|
|1.0296194171811581| 1.410986973710262|            2018|            0|
| 5.225746673713202| 6.914730892718563|            2008|            1|
| 5.181783550292085| 6.572841826824917|            2014|            1|
+------------------+------------------+----------------+-------------+
only showing top 5 rows


In [0]:
features_df.write.mode("overwrite").saveAsTable("default.ml_features_movies")
